In [1]:
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from nilearn import datasets, image, plotting
from nilearn.masking import apply_mask
from nilearn.signal import clean
sys.path.append('..')
from utils.data import Subject, load_participant_list

In [2]:
sub = '02'
run = 'test'
tr = 2.33384

In [3]:
base_dir = '/home/ubuntu/data/learning-habits'
bids_dir = "/home/ubuntu/data/learning-habits/bids_dataset/derivatives/fmriprep-24.0.1"

sub_ids = load_participant_list(base_dir)

In [4]:
# Load a V1 mask
# Example using Juelich atlas (probabilistic)
juelich = datasets.fetch_atlas_juelich('maxprob-thr50-1mm')
labels = juelich.labels
# Suppose index 8 is V1 left
v1_index = 48
print(labels[v1_index])  # Look for 'Primary visual cortex' or 'V1'

v1_mask_img = image.math_img("img == %d" % v1_index, img=juelich.maps)

GM Visual cortex V1 BA17


In [5]:
#plotting.plot_roi(v1_mask_img, title='V1 mask', display_mode='mosaic')

In [6]:
subject = Subject(base_dir, sub, include_modeling=False, include_imaging=True, bids_dir=bids_dir)

In [7]:
# Load fMRI volume
img_path = subject.img.get(run)
fmri_img = image.load_img(img_path)

# resample mask
v1_mask_img = image.resample_to_img(v1_mask_img, fmri_img, interpolation='nearest')

In [28]:
anat_path = (
    subject.get_brain_mask(run)
    .replace('/func/', '/anat/')[:106] + 
    'run-1_desc-preproc_T1w.nii.gz'
)
anat_img = image.load_img(anat_path)

In [ ]:
brain_mask = image.load_img(subject.get_brain_mask(run))

In [8]:
fmri_img.affine

array([[   3. ,    0. ,    0. ,  -78. ],
       [   0. ,    3. ,    0. , -114. ],
       [   0. ,    0. ,    3.5,  -78. ],
       [   0. ,    0. ,    0. ,    1. ]])

In [13]:
brain_mask.affine

array([[   3. ,    0. ,    0. ,  -78. ],
       [   0. ,    3. ,    0. , -114. ],
       [   0. ,    0. ,    3.5,  -78. ],
       [   0. ,    0. ,    0. ,    1. ]])

In [30]:
anat_img.affine

array([[   1.        ,    0.        ,    0.        ,  -85.        ],
       [   0.        ,    0.93969262,   -0.34202015,  -75.90440369],
       [   0.        ,    0.34202015,    0.93969262, -162.77752686],
       [   0.        ,    0.        ,    0.        ,    1.        ]])

In [ ]:
# Load events
events = getattr(subject, run).events

In [ ]:
# Load confounds
confounds, sample_mask = subject.load_confounds(run, motion_type='basic')
physio_regressors = subject.load_physio_regressors(run)
confounds = confounds.join(physio_regressors)

In [ ]:
masked_img = apply_mask(fmri_img, v1_mask_img)
cleaned_masked_img = clean(masked_img, confounds=confounds, t_r=tr, detrend = True,
                           standardize='zscore_sample')

/tmp/ipykernel_295346/2199374517.py:2: DeprecationWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  cleaned_masked_img = clean(masked_img, confounds=confounds, t_r=tr, detrend = True,
